In [1]:
import numpy as np
from agents.random_agent import random_policy
from citylearn.citylearn import CityLearnEnv
import wandb
import random
import torch
import numpy as np
from collections import namedtuple, deque
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Constants:
    episodes = 3
    schema_path = './data/citylearn_challenge_2022_phase_1/schema.json'

def action_space_to_dict(aspace):
    """ Only for box space """
    return { "high": aspace.high,
             "low": aspace.low,
             "shape": aspace.shape,
             "dtype": str(aspace.dtype)
    }

def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    building_info = env.get_building_information()
    building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {"action_space": action_space_dicts,
                "observation_space": observation_space_dicts,
                "building_info": building_info,
                "observation": observations }
    return obs_dict

In [3]:
env = CityLearnEnv(schema=Constants.schema_path)

['aicrowd.json', 'starter.ipynb', 'local_evaluation.py', 'train.py', 'data', '.ipynb_checkpoints', 'ddpg_Analysis.ipynb', 'agents', 'submit.sh', 'requirements.txt', 'drl_algo', 'README.md', 'rewards', 'apt.txt', 'going-below-1-0-score-with-stablebaseline3.ipynb', 'docs']


In [6]:
import gym
class EnvCityGym(gym.Env):
    """
    Env wrapper coming from the gym library.
    """
    def __init__(self, env):
        self.env = env

        # get the number of buildings
        self.num_buildings = len(env.action_space)

        # define action and observation space
        self.action_space = gym.spaces.Box(low=np.array([-1] * self.num_buildings), high=np.array([1] * self.num_buildings), dtype=np.float32)

        # # define the observation space
        # self.observation_space = gym.spaces.Box(low=np.array([0] * len_tot_index), high=np.array([1] * len_tot_index), dtype=np.float32)

        # TO THINK : normalize the observation space

    def reset(self):
        obs_dict = env_reset(self.env)
        obs = self.env.reset()

        observation = self.get_observation(obs)

        return observation

    # def get_observation(self, obs):
    #     """
    #     We retrieve new observation from the building observation to get a proper array of observation
    #     Basicly the observation array will be something like obs[0][index_commun] + obs[i][index_particular] for i in range(5)

    #     The first element of the new observation will be "commun observation" among all building like month / hour / carbon intensity / outdoor_dry_bulb_temperature_predicted_6h ...
    #     The next element of the new observation will be the concatenation of certain observation specific to buildings non_shiftable_load / solar_generation / ...  
    #     """
        
    #     # we get the observation commun for each building (index_commun)
    #     observation_commun = [obs[0][i]/n for i, n in zip(index_commun, normalization_value_commun)]
    #     observation_particular = [[o[i]/n for i, n in zip(index_particular, normalization_value_particular)] for o in obs]

    #     observation_particular = list(itertools.chain(*observation_particular))
    #     # we concatenate the observation
    #     observation = observation_commun + observation_particular

        return observation

    def step(self, action):
        """
        we apply the same action for all the buildings
        """
        # reprocessing action
        action = [[act] for act in action]

        # we do a step in the environment
        obs, reward, done, info = self.env.step(action)

        observation = self.get_observation(obs)

        return observation, sum(reward), done, info
        
    def render(self, mode='human'):
        return self.env.render(mode)


env = EnvCityGym(env)
env.possible_agents

/home/frozenwolf/miniconda3/envs/citylearn/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


NameError: name 'len_tot_index' is not defined

In [4]:
Transition_DDPG = namedtuple('Transition', ('state', 'next_state', 'action', 'reward', 'done'))
class DDPG_Memory(object):
    def __init__(self,capacity):
        """
        Replay buffer for DDPG
        INPUT  : state,next_state,action,reward and done
        OUTPUT : batch of off-polocy experience
        """
        self.memory = deque(maxlen=capacity)
        self.capacity = capacity

    def push(self, state, next_state, action, reward, done):
        self.memory.append(Transition_DDPG(state, next_state, action, reward, done))

    def sample(self, batch_size):
        transitions = random.sample(self.memory, batch_size)
        batch = Transition_DDPG(*zip(*transitions))
        return batch

    def __len__(self):
        return len(self.memory)


In [7]:
env.action_space[0].shape

(1,)